<a id='top'></a>
# A Quantum Approach for Job Shop Scheduling #

## Table of Contents
1. [Introduction](#job_introduction)
2. [Detailed problem description](#prob_description)
    <br>2.a. [The constraints](#the_constraints)
    <br>2.b. [The final objective function](final_objective_function)
3. [Coding up the problem](#code_problem)
    <br>3.a. [Problem parameters](#problem_parameters)
    <br>3.b. [Constructing the objective function](#const_objective_function)
4. [Quantum Computing Solution with the QC Ware API](#quantum_solution)
    <br>4.a. [Execution via IBM's quantum environment](#quantum_env_IBM)
    <br>4.b. [Execution via Google's quantum environment](#quantum_env_google)
    <br>4.c. [Execution via D-Wave's hardware framework](#hardware_framework_dwave)
    <br>4.d. [Bonus: the purely classical brute force solver as a debugging tool](#brute_force_debugging_tool)
  

## 1. Introduction <a id="job_introduction"></a>

The job shop scheduling problem is the problem of optimally allocating a set of jobs to a set of workers in such a way that the makespan is minimized.  The makespan of a project is the total time required to complete all jobs.

***
# 2. Detailed problem description <a id="prob_description"></a>

While there are many different ways of formally modelling the problem, here we focus on a quadratic unconstrained binary optimization (QUBO) model of the problem.  This allows us to treat the problem both on universal quantum computers as well as on special purpose quantum annealers.  Before we proceed, we briefly mention that for Operations Research professionals wondering how to formulate their problems as a QUBO, the paper <a href="https://arxiv.org/pdf/1302.5843.pdf" style="text-decoration: none;">Ising formulations of many NP problems</a>, by Andrew Lucas, 2014, gives <em>Ising</em> formulations of many NP problems.  Given that Ising and QUBO problems are equivalent under the appropriate transformation, one can in turn use these to obtain the QUBO forms.

Let us now formally define the problem.  Suppose we are interested in completing $N$ jobs,  and we have access to $m$ workers who can complete these jobs.  Furthermore, assume each job $i\in\{1,\dots,N\}$ takes time $L_i$ to complete.  While other formulations are certainly possible, here we make the simplifying assumption that a given job will take each worker the same amount of time, i.e., $L_i$ is independent of the worker $\alpha\in\{1,\dots,m\}$.  Let $V_{\alpha}$ be the set of jobs assigned to worker $\alpha$.  Define the length of the worker $\alpha$ to be 
\begin{equation}
M_{\alpha}:=\sum_{i\in V_{\alpha}}L_i.
\end{equation}
Our goal is to assign jobs to workers in such a way that $\text{max}(M_{\alpha})$ is minimized.  This means that the makespan of the project is minimized.

For each $i,\alpha$, we introduce a binary decision variable $x_{i,\alpha}$ which equals 1 if job $i$ is assigned to worker $\alpha$, and 0 otherwise.  Without loss of generality, assume that $M_{1}\geq M_{\alpha}$.  Under this assumption, the goal would be to minimize, over all feasible assignments of the decision variables, the term
\begin{equation}
H_{B}:=BM_{1}=B\sum_{i=1}^{N}L_ix_{i,1},
\end{equation}
where $B\in(0,\infty]$ is a scaling parameter we choose later.

## 2.a. The constraints <a id="the_constraints"></a>

What do we mean by "feasible" assignments?  We are only interested in assigmnets of the decision variables which satisfy the following two constraints:
\begin{equation}
\sum_{\alpha=1}^{m}x_{i,\alpha}=1 \; \forall \; i\in\{1,\dots,N\}, \tag{Constraint 1}
\end{equation}
and
\begin{equation}
M_{1}\geq M_{\alpha} \; \forall \; \alpha\in\{2,\dots,m\} \tag{Constraint 2}.
\end{equation}
The first constraint can be interpreted as the requirement that each job be assigned to precisely one worker.  The second constraint can be interpreted as the requirement that no worker have an assignment of jobs which takes longer to complete than those of worker 1.  We will enforce these constraints by including the appropriate penalty functions in the final objective function.

### Constraint 1: Ensuring each job is assigned

Let us start with Constraint 1, which is simpler to take care of.  To do so, we simply need to include a term of the form
\begin{equation}
H^{1}_{A}:=A\sum_{i=1}^{N}\left(1-\sum_{\alpha=1}^{m}x_{i,\alpha}\right)^2,
\end{equation}
where $A\in(0,\infty)$ is a scaling parameter we choose later.  Note that $H^{1}_{A}\geq 0$, and it equals 0 (i.e., it's minimum) if and only if the assignment satisfies Constraint 1.

### Constraint 2: Ensuring worker 1 has more work than any other worker

The second constraint is a bit trickier to enforce.  First, introduce an additional binary decision variable $y_{n,\alpha}$ for each $\alpha\in\{2,\dots,m\}, n\in\{0,\dots,\lfloor\log_{2}\mathcal{M}\rfloor\}$.  We will elaborate on $\mathcal{M}$ shortly.  For now, consider the expression
\begin{equation}
H^{2}_{A}:=A\sum_{\alpha=2}^{m}\left(\sum_{n=0}^{\lfloor\log_{2}\mathcal{M}\rfloor}2^ny_{n,\alpha}+\sum_{i=1}^{N}L_{i}(x_{i,\alpha}-x_{i,1})\right)^2.
\end{equation}
Note that $H^{2}_{A}\geq0$, and it equals 0 (i.e., its minimum) whenever \begin{equation}
\sum_{n=0}^{\lfloor\log_{2}\mathcal{M}\rfloor}2^ny_{n,\alpha}+\sum_{i=1}^{N}L_{i}(x_{i,\alpha}-x_{i,1})\overset{!}{=}0 \; \forall \; \alpha\in\{2,\dots,m\}.
\end{equation}
Because $\sum_{n=0}^{\lfloor\log_{2}\mathcal{M}\rfloor}2^ny_{n,\alpha}\geq 0$, the above equation can hold only if $\sum_{i=1}^{N}L_{i}(x_{i,\alpha}-x_{i,1})=M_{\alpha}-M_{1}\leq0 \; \forall \; \alpha\in\{2,\dots,m\}$, i.e., precisely if Constraint 2 is satisfied!  More specifically, assuming $M_{\alpha}-M_{1}\leq0$, note that the above equation holds whenever the $\{y_{n,\alpha}\}$ are the digits in the binary expansion of $M_{\alpha}-M_{1}$.  This means that $\mathcal{M}$ needs to be large enough so that $\lfloor\log_{2}\mathcal{M}+1\rfloor$ binary digits suffice to express $M_{\alpha}-M_{1}$.  That holds whenever $\mathcal{M}\overset{!}{\geq} M_{\alpha}-M_{1}$.  Because $M_{\alpha}-M_{1}\leq N\max_{i}L_{i}$, we can ensure this by choosing $\mathcal{M}=N\max_{i}L_{i}$.

## 2.b. The final objective function <a id="final_objective_function"></a>

We can now construct our final objective function by adding the terms we have constructed above.
\begin{equation}
H:=H^{1}_{A}+H^{2}_{A}+H_{B}=
\end{equation}
\begin{equation}
A\sum_{i=1}^{N}\left(1-\sum_{\alpha=1}^{m}x_{i,\alpha}\right)^2+A\sum_{\alpha=2}^{m}\left(\sum_{n=0}^{\lfloor\log_{2}\mathcal{M}\rfloor}2^ny_{n,\alpha}+\sum_{i=1}^{N}L_{i}(x_{i,\alpha}-x_{i,1})\right)^2+B\sum_{i=1}^{N}L_ix_{i,1}.
\end{equation}

In order to guarantee that that the ground state encodes the solution, we require $0<B\max{(L_i)}<A$.  The number of logical variables required is $mN+(m-1)\lfloor1+\log\mathcal{M}\rfloor$.

***
# 3. Coding up the problem <a id="code_problem"></a>

## 3.a. Problem parameters <a id="problem_parameters"></a>

We are now ready to start writing the code we'll use to treat this problem.  We start by importing the native python module `random`, which provides the functionality to generate pseduo-random numbers; we'll also use the built-in module `pprint` to pretty-print some results.

In [1]:
from qcware import forge
# this line is for internal tracking; it is not necessary for use!
forge.config.set_environment_source_file('job_shop_scheduling.ipynb')

import random
import pprint

# for consistency, we'll seed the random number to a predetermined value
random.seed(42)

We choose the number of jobs.

In [2]:
N = 3

Whe choose the number of workers.

In [3]:
m = 3

We choose the time it takes to complete each job.

In [4]:
# the following code would generate random job lengths; 

lengths = list(range(1, 10+1))
L = {}
for i in range(1, N+1):
    L[i] = random.choice(lengths)

for (k, v) in L.items():
    print("Job {0} would take {1} units of time".format(k, v))

Job 1 would take 2 units of time
Job 2 would take 1 units of time
Job 3 would take 5 units of time


Here the choose the scaling parameters $A$ and $B$.  Recall that we can guarantee that the global ground state encodes the solution by choosing $A>B\max_{i=1}^{N}(L_i)$.

In [5]:
# Penalty terms
B = 1
A = B * max(L.values())

## 3.b. Constructing the objective function <a id="const_objective_function"></a>

We are now ready to start coding up the QUBO. We will use the Python package `qubovert` that we developed to aid in binary optimization formulations. Let's install it and import `boolean_var`. We will use it to create a Polynomial Constrained Boolean Optimization (PCBO) object.

In [6]:
from qubovert import boolean_var

Now we will create the $x_{i,\alpha}$ variables. We'll create a dictionary `x` such that `x[(i, alpha)]` will return the variable for $x_{i,\alpha}$.

In [7]:
x = {
    (i, alpha): boolean_var('x(%d, %d)' % (i, alpha)) 
    for i in range(1, N+1) for alpha in range(1, m+1)
}

Now we create the $M_\alpha$'s. Recall that
$$M_{\alpha} = \sum_{i=1}^N L_i x_{i, \alpha}.$$
We create a dictionary `M` such that `M[alpha]` returns $M_{\alpha}$.

In [8]:
M = {
    alpha: sum(L[i] * x[(i, alpha)] for i in range(1, N+1)) 
    for alpha in range(1, m+1)
}

Next we will encode $H_B = B M_1$. 

In [9]:
H = B * M[1]

pprint.pprint(H)

{('x(1, 1)',): 2, ('x(2, 1)',): 1, ('x(3, 1)',): 5}


Now we encode the first constraint,
$$\sum_{\alpha=1}^{m}x_{i,\alpha}=1 \quad \forall \; i\in\{1,\dots,N\},$$
or equivalently
$$\sum_{\alpha=1}^{m}x_{i,\alpha}-1=0 \quad \forall \; i\in\{1,\dots,N\}.$$
Recall that we enforce this constraint with the penalty factor $A$.

In [10]:
for i in range(1, N+1):
    H.add_constraint_eq_zero(
        sum(x[(i, alpha)] for alpha in range(1, m+1)) - 1,
        lam=A
    )
    
pprint.pprint(H)

{(): 15,
 ('x(1, 1)',): -3,
 ('x(1, 1)', 'x(1, 3)'): 10,
 ('x(1, 2)',): -5,
 ('x(1, 2)', 'x(1, 1)'): 10,
 ('x(1, 2)', 'x(1, 3)'): 10,
 ('x(1, 3)',): -5,
 ('x(2, 1)',): -4,
 ('x(2, 1)', 'x(2, 2)'): 10,
 ('x(2, 2)',): -5,
 ('x(2, 3)',): -5,
 ('x(2, 3)', 'x(2, 1)'): 10,
 ('x(2, 3)', 'x(2, 2)'): 10,
 ('x(3, 1)', 'x(3, 2)'): 10,
 ('x(3, 2)',): -5,
 ('x(3, 3)',): -5,
 ('x(3, 3)', 'x(3, 1)'): 10,
 ('x(3, 3)', 'x(3, 2)'): 10}


Now we encode the second constraint,
$$M_{1}\geq M_{\alpha} \quad \forall \; \alpha\in\{2,\dots,m\},$$
or equivalently
$$M_{1} - M_{\alpha}\geq 0 \quad \forall \; \alpha\in\{2,\dots,m\}.$$
Recall that we enforce this constraint with the penalty factor $A$.

In [11]:
for alpha in range(2, m+1):
    H.add_constraint_ge_zero(M[1] - M[alpha], lam=A)

# print(H)  # large output

Notice that ancilla variables, labeled with `__a`, were automatically added to enforce the constraints.

We note that the degree of `H` is only 2 and is therefore already a QUBO.

In [12]:
H.degree

2

We will still do the standard procedure since it works even if the degree of `H` is greater than 2.

In [13]:
qubo = H.to_qubo()
# print(qubo)  # large output

Now the QUBO formulation is complete! We can summarize what we have done in this section in the following schematic:

![](https://files.slack.com/files-pri/T24940PQV-FH78R2A07/job_shop_scheduling_schematic.png?pub_secret=3ef8d30f11)

***
## 4. Quantum Computing Solution with the QC Ware API <a id="quantum_solution"></a>

Using QC Ware's API, we can solve this problem with the `optimize_binary()` function.  With the `optimize_binary()` function, the user has the option to choose from a number of different backend solvers.  Without having to change the way the QUBO is input, QC Ware's tools then automatically formulate the problem in a way that is suitable for the selected solver's corresponding software and hardware environment.  This allows the user to explore with minimal hassel the backend that might be most well-suited for her application.

Let us take a look at a few different solvers.  We start by importing the `qcware` module and entering your API key as necessary (you can find your API key on your dashboard on [Forge](https://forge.qcware.com)).

In [14]:
import qcware.types

We now have a formulated problem, it only remains to choose a backend solver to execute on.

Let's create the function `solve` that calls Forge and interprets the solution. Notice that we use the `H.convert_solution` method to convert the solution of the QUBO to the solution of the PCBO. We also use the `H.remove_ancilla_from_solution` method to remove all the ancilla information from the solution since they were only added to help enforce constraints. Finally, the `H.is_solution_valid` method checks to see if the proposed solution satisfies all of the constraints.

In [15]:
def solve_with(backend):
    poly = qcware.types.optimization.PolynomialObjective(
    polynomial=qubo.Q,
    num_variables=qubo.num_binary_variables,
    domain='boolean'
    )
    problem = qcware.types.optimization.BinaryProblem(objective=poly)
    response = forge.optimization.optimize_binary(instance=problem, backend=backend, dwave_num_reads=50)
    
    qubo_solution = response.lowest_value_bitstring
    pcbo_solution = H.convert_solution(qubo_solution)
    
    if H.is_solution_valid(pcbo_solution):
        print("solution is valid\n")
    else:
        print("solution is invalid\n")
        
    print("objective value:", H.value(pcbo_solution))
    pprint.pprint(H.remove_ancilla_from_solution(pcbo_solution))

Notice that the problem has 17 binary variables.

In [16]:
H.num_binary_variables

17

Therefore, we can use the brute force solver to test our problem formulation!  The brute-force solver (what you get when solving a binary problem with the `qcware/cpu` backend) can take quite a while, so we'll need to either set the default timout to something large or use the `async`/`await` keywords as in the `Retrieving_Long_Task_Results.ipynb` notebook.  Here, let's just set the default timeout to something large like 5 minutes.

In [17]:
forge.config.set_client_timeout(5*60)
solve_with('qcware/cpu')

solution is valid

objective value: 5
{'x(1, 1)': 0,
 'x(1, 2)': 0,
 'x(1, 3)': 1,
 'x(2, 1)': 0,
 'x(2, 2)': 0,
 'x(2, 3)': 1,
 'x(3, 1)': 1,
 'x(3, 2)': 0,
 'x(3, 3)': 0}


We see that the nonzero entries are `x(3, 1)`, `x(1, 3)`, and `x(2, 3)`. Recall that the first number corresponds to the index `i` indicating the job, and the second number corresponds to `alpha` indicating the worker. Therefore, we see that a optimal solution is for worker 1 to cover job 3 and for worker 3 to cover jobs 1 and 2. The objective value is 5, indicating that the total length of the schedule is 5.

### 4.a. Execution via IBM's quantum environment <a id="quantum_env_IBM"></a>

The first solver we explore is `ibm_sw_qaoa`, which is an implementation of the quantum approximate optimization aglorithm (QAOA) [<sup>2</sup>](https://arxiv.org/abs/1411.4028) for (classical) energy minimization problems of the Ising model, developed using IBM's Qiskit [<sup>3</sup>](https://qiskit.org) Aqua [<sup>4</sup>](https://qiskit.org/aqua) software framework.  We brielfy recall that any Ising solver (such as `ibm_sw_qaoa`) can also be used for QUBOs due to the mathematical equivalence between QUBOs and Ising problems under the appropriate transformation.  This in turns allows us to solve our QUBO formulation of the portfolio optimization problem.

*Note*: The software QAOA solver runs as a quantum circuit emulator and can take a very long time (upwards of an hour) to return a result, so the following code is commented out; uncomment if you have some time to burn!

In [18]:
# solve('ibm_sw_qaoa')

### 4.b. Execution via Google's quantum environment <a id="quantum_env_google"></a>

The second solver we explore is `google_qaoa`, which is an implementaton of the QAOA for (classical) energy minimization problems of the Ising model, developed using Google's Cirq  [<sup>5</sup>](https://ai.googleblog.com/2018/07/announcing-cirq-open-source-framework.html) software framework.  As we can see below, switching to this different backend is done simply by changing the value of the string corresponding desired backend solver.  Similarly, this solver takes a long time to run, so it's commented out for now!

In [19]:
# solve('google_qaoa')

### 4.c. Execution via D-Wave's hardware <a id="hardware_framework_dwave"></a>

Finally, we will solve the problem with D'Wave's quantum annealer.

Again, as we can see below, despite the notably different architectures of universal quantum computers and special purpose quantum annealers, switching to this different backend is done simply by changing the value of the string corresponding desired backend solver.

In [20]:
solve_with('dwave/2000q')

solution is invalid

objective value: 15
{'x(1, 1)': 0,
 'x(1, 2)': 0,
 'x(1, 3)': 0,
 'x(2, 1)': 0,
 'x(2, 2)': 0,
 'x(2, 3)': 1,
 'x(3, 1)': 1,
 'x(3, 2)': 1,
 'x(3, 3)': 0}


Note: If you are viewing this on GitHub and want to access QC Ware's API to execute the notebook you will need an API key. Please reach out stanford.schor@qcware.com to request access.
<br><a href="#top">Back to Table of Contents</a>